In [1]:
import torch.nn as nn
import torch

/home/kbh/.conda/envs/dnn/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
class SEBlock(nn.Module) :
    def __init__(self,in_channel,ratio=16):
        super().__init__()
        
        size_se = int(in_channel/ratio)
       
        self.pool = torch.nn.AdaptiveAvgPool2d((1,1))
            # Squeeze
        self.squeeze = nn.Linear(in_channel,size_se)
        self.relu = nn.ReLU()
            # Excitation
        self.excitation = nn.Linear(size_se,in_channel)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        # x : [B,C,F,T]
        
        
        x_ = self.pool(x)
        x_ = x_.permute(0,2,3,1)
        
        x_ = self.squeeze(x_)
        x_ = self.relu(x_)
        x_ = self.excitation(x_)
        x_ = self.sigmoid(x_)
        x_ = x_.permute(0,3,1,2)
        x = x*x_
        
       
        return x

In [29]:
x = torch.rand(2,64,32,16)
print(x.shape)
m = SEBlock(64)

y = m(x)
print(y.shape)

torch.Size([2, 64, 32, 16])
torch.Size([2, 1, 1, 64])
torch.Size([2, 64, 32, 16])
